<a href="https://colab.research.google.com/github/IFuentesSR/SAMSARA_UAI/blob/main/Classification_disturbances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Code for getting secondary classification and disentangling deforestation from other forest disturbances

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize(project='your_project') #use your credentials

In [ ]:
import numpy as np
import folium

In [ ]:
tree = 150

In [ ]:
def to_year(img):
    year2000 = 730485
    year = img.subtract(year2000).divide(365)
    return ee.Image(2000).add(ee.Image(year))


def to_day(img):
    year = img.subtract(2016).multiply(365)
    return year


#### Defining inputs for classification

In [ ]:
'''import your input images saved, here I used those that I saved'''

s1 = ee.Image('users/ignisfausto/rvi_iqr')
dem = ee.Image("MERIT/DEM/v1_0_3")
LC = ee.Image("users/ignaciofuentessanroman/LC_CHILE_2014_b")
mask = LC.gt(210).And(LC.lt(300)).Or(LC.gt(400).And(LC.lt(460)))

mag1 = ee.Image('users/ignisfausto/ccdc_magnitude_001083_095')
mag2 = ee.Image('users/ignisfausto/ccdc_magnitude_001084_095')
mag3 = ee.Image('users/ignisfausto/ccdc_magnitude_001085_095')
mag4 = ee.Image('users/ignisfausto/ccdc_magnitude_233083_095')
mag5 = ee.Image('users/ignisfausto/ccdc_magnitude_233084_095')
mag6 = ee.Image('users/ignisfausto/ccdc_magnitude_233085_095')
magMosaic = ee.ImageCollection([mag1, mag2, mag3, mag4, mag5, mag6]).mosaic()

bks1 = ee.Image('users/ignisfausto/ccdc_tStart_001083_095')
bks2 = ee.Image('users/ignisfausto/ccdc_tStart_001084_095')
bks3 = ee.Image('users/ignisfausto/ccdc_tStart_001085_095')
bks4 = ee.Image('users/ignisfausto/ccdc_tStart_233083_095')
bks5 = ee.Image('users/ignisfausto/ccdc_tStart_233084_095')
bks6 = ee.Image('users/ignisfausto/ccdc_tStart_233085_095')
bksMosaic = ee.ImageCollection([bks1, bks2, bks3, bks4, bks5, bks6]).mosaic()

glcm = ee.Image('users/ignaciofuentessanroman/ccdc_glcm_mgs_30_7_2')
bks_sdev = ee.Image('users/ignaciofuentessanroman/bks50_in_ccdc2').select('first_stdDev')
mgs_sdev = ee.Image('users/ignaciofuentessanroman/mgs50_in_ccdc2').select('first_stdDev')

regions = ee.FeatureCollection('users/ignisfausto/regions5_7')

In [ ]:
bksMosaic = to_year(bksMosaic)
mask = bksMosaic.gte(2016)
bksMosaic = bksMosaic.updateMask(mask)

mask2 = magMosaic.lt(0)
mask2 = ee.Image(0).addBands(mask2.select([0,1,2,3]))

bksMosaic = bksMosaic.updateMask(mask).updateMask(mask2)
bksMosaic = bksMosaic.select([1,2,3,4])
magMosaic = magMosaic.updateMask(mask.select([1,2,3,4]).addBands(ee.Image(0))).updateMask(mask2.select([1,2,3,4]).addBands(ee.Image(0)))

bksMosaic2 = bksMosaic.reduce(ee.Reducer.firstNonNull())
magMosaic2 = magMosaic.reduce(ee.Reducer.firstNonNull())

##### Visualisation of dates and magnitudes of changes

In [ ]:
bks_img = bksMosaic2.clip(regions).getMapId({'min': 2016, 'max': 2022.5, 'palette':'FF0000, FFFF00, 00FF00, 00FFFF, 0000FF'})
mgs_img = magMosaic2.clip(regions).getMapId({'min': -1, 'max': 1, 'palette':'FF0000, FFFFFF, 0000FF'})
centroid = regions.geometry().centroid().coordinates().getInfo()[::-1]
map = folium.Map(location=centroid, zoom_start=6)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=bks_img['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='break_dates',
  ).add_to(map)
folium.TileLayer(
    tiles=mgs_img['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='break_magnitudes',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

#### Ground truth collection for model training and validation

In [ ]:
poly = ee.FeatureCollection("users/ignaciofuentessanroman/PolValConsolidadosBosquePlantacion")
poly2 = poly.filter(ee.Filter.Or(ee.Filter.gte('ID', 370), ee.Filter.lte('ID',328)))

In [ ]:
fires = ee.Geometry.MultiPolygon(
    [[[[-71.53702009863046, -33.110794276712355],
       [-71.5365051144996, -33.11180077494024],
       [-71.53538931554941, -33.1120164516317],
       [-71.53444517797617, -33.10971587295818],
       [-71.53641928381113, -33.11007534234702]]],
     [[[-71.59924734777597, -33.132287696217126],
       [-71.59744490331796, -33.133437697468054],
       [-71.59684408849863, -33.13185644186388],
       [-71.59856070226816, -33.13085017346732],
       [-71.60027731603769, -33.13041891205213],
       [-71.60027731603769, -33.13178456593237]]],
     [[[-71.58165205663828, -33.13386894405297],
       [-71.58027876562265, -33.13473143086677],
       [-71.57907713598398, -33.13329394813561],
       [-71.57967795080332, -33.13092205016389],
       [-71.58130873388437, -33.13257519794227]]],
     [[[-71.53410185522226, -33.16020690514725],
       [-71.53461683935312, -33.15855427791913],
       [-71.53564680761484, -33.15848242385489],
       [-71.53654802984384, -33.16009912605998],
       [-71.53611887640146, -33.16232653360874],
       [-71.53538931554941, -33.16365576599389],
       [-71.5330289716163, -33.16494905383778],
       [-71.53238524145273, -33.16455388457613],
       [-71.53114069646982, -33.163727615819035],
       [-71.53187025732187, -33.160709872469546],
       [-71.53405893987802, -33.15873391282216],
       [-71.53362978643564, -33.16088950295693]]]])


### set of polygon IDs needed or wrongly classified by Andrés
non_fires = [374, 375, 376, 393, 394, 406, 424]

non_stable = [81, 83]

non_defo = [85, 90, 113, 115, 116, 117, 118, 119, 128,
            130, 138, 237, 236, 235, 232, 230, 127, 114, 87]

stable_list = [85, 90, 113, 115, 116, 117, 118, 119,
               130, 237, 236, 235, 232, 230, 114]

defo_list = [330, 334, 349, 348, 351, 353, 354, 358, 362,
             369, 366, 347, 367, 365, 364, 349, 348, 359,
             357, 356, 355, 350]


In [ ]:
def set_fire(f):
    return f.set('change', 0)

def set_drought(f):
    return f.set('change', 0)

def set_stable(f):
    return f.set('change', 0)

def set_defo(f):
    return f.set('change', 1)

def set_id(f):
    return f.set('id', f.id())


fire = poly2.filter(ee.Filter.gte('ID', 370)).merge(fires).filter(ee.Filter.inList('ID', non_fires).Not())
fire = fire.map(set_fire)

drought = poly2.filter(ee.Filter.And(ee.Filter.gte('ID', 238), ee.Filter.lte('ID', 328)))
drought = drought.map(set_drought)

stable = poly2.filter(ee.Filter.eq('NAME', 'estable')).filter(ee.Filter.inList('ID', non_stable).Not())
stable2 = poly.filter(ee.Filter.inList('ID', stable_list))
stable = stable.merge(stable2).map(set_stable)

defo = poly2.filter(ee.Filter.eq('NAME', 'tala')).filter(ee.Filter.inList('ID', non_defo).Not())
defo2 = poly.filter(ee.Filter.inList('ID', defo_list))
defo = defo.merge(defo2).map(set_defo)

collection = fire.merge(drought).merge(stable).merge(defo).map(set_id)

#### Setting training image for model

In [ ]:
glcm_bands = ['first_asm', 'first_contrast', 'first_idm', 'first_ent', 'first_savg']

glcm_in = ee.ImageCollection([ee.Image(ee.List.repeat(-999, len(glcm_bands)).getInfo()).toFloat().rename(glcm_bands),
                              glcm.select(glcm_bands)]).mosaic()

mgs_dev_in = ee.ImageCollection([ee.Image([-999]).toFloat().rename(['mgs_dev']),
                                 mgs_sdev.rename(['mgs_dev'])]).mosaic()



bks_dev_in = ee.ImageCollection([ee.Image([-999]).toFloat().rename(['bks_dev']),
                                 bks_sdev.rename(['bks_dev'])]).mosaic()



inputImg = magMosaic2.select(0).addBands(glcm_in).addBands(mgs_dev_in).addBands(bks_dev_in) \
    .addBands(dem).addBands(ee.Terrain.slope(dem)).addBands(s1.select(['iqr','rvi_p25','rvi_p75']))

classBands = inputImg.bandNames()

#### 10-Fold cross validation on polygons

In [ ]:
collection = collection.randomColumn(seed=3)
values = ee.List.sequence(0, 0.9, 0.1)

In [ ]:
def classification(c, p):
    val0 = ee.Number(c)
    val1 = val0.add(0.2)
    train = collection.filter(ee.Filter.Or(ee.Filter.lte('random', val0), ee.Filter.gt('random', val1)))
    vali = collection.filter(ee.Filter.And(ee.Filter.gt('random', val0), ee.Filter.lte('random', val1)))
    sample_train = inputImg.sampleRegions(collection=train,
                                          properties=['change'],
                                          scale=30,
                                          geometries=True)

    sample_vali = inputImg.sampleRegions(collection=vali,
                                         properties=['change'],
                                         scale=30,
                                         geometries=True)

    classifier = ee.Classifier.smileRandomForest(tree)
    trainedRF = classifier.train(sample_train, 'change', classBands)
    trainCM = trainedRF.confusionMatrix()
    validationDS = sample_vali.classify(trainedRF)
    validationCM = validationDS.errorMatrix('change', 'classification')
    classified = inputImg.classify(trainedRF)
    return ee.List(p).add(ee.List([trainCM.accuracy(), trainCM.kappa(), trainCM.fscore(), trainCM.consumersAccuracy(), trainCM.producersAccuracy(),
                               validationCM.accuracy(), validationCM.kappa(), validationCM.fscore(), validationCM.consumersAccuracy(), validationCM.producersAccuracy(),
                               classified]))


iter = values.iterate(classification, ee.List([])).getInfo()

In [ ]:
kappa_validation = [n[6] for n in iter]
accuracy_validation = [n[5] for n in iter]
print(np.mean(kappa_validation), np.mean(accuracy_validation))

0.8674033907988603 0.9429803275225372


##### Visualisation of deforestation

In [ ]:
classBands = inputImg.bandNames()
classifier = ee.Classifier.smileRandomForest(tree)
sample_train = inputImg.sampleRegions(collection=collection,
                                          properties=['change'],
                                          scale=30,
                                          geometries=True)
trainedRF = classifier.train(sample_train, 'change', classBands)
classified_img = inputImg.classify(trainedRF)

In [ ]:
classified_img2 = ee.Image(0).where(classified_img.eq(1), classified_img).multiply(mask)
mapid_img = classified_img2.focal_mean(2).updateMask(classified_img2).getMapId({'min': 0, 'max': 1, 'palette':'FFFFFF, FF0000'})
centroid = collection.geometry().centroid().coordinates().getInfo()[::-1]
map = folium.Map(location=centroid, zoom_start=6)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid_img['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='defo',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

#### Classification including other classes (drought, fires)

##### Setting polygons

In [ ]:
def set_fire(f):
    return f.set('change', 2)

def set_drought(f):
    return f.set('change', 3)

def set_stable(f):
    return f.set('change', 0)

def set_defo(f):
    return f.set('change', 1)

def set_id(f):
    return f.set('id', f.id())


fire2 = poly2.filter(ee.Filter.gte('ID', 370)).merge(fires).filter(ee.Filter.inList('ID', non_fires).Not())
fire2 = fire2.map(set_fire)

drought2 = poly2.filter(ee.Filter.And(ee.Filter.gte('ID', 238), ee.Filter.lte('ID', 328)))
drought2 = drought2.map(set_drought)

stable3 = poly2.filter(ee.Filter.eq('NAME', 'estable')).filter(ee.Filter.inList('ID', non_stable).Not())
stable4 = poly.filter(ee.Filter.inList('ID', stable_list))
stable2 = stable3.merge(stable4).map(set_stable)

defo3 = poly2.filter(ee.Filter.eq('NAME', 'tala')).filter(ee.Filter.inList('ID', non_defo).Not())
defo4 = poly.filter(ee.Filter.inList('ID', defo_list))
defo2 = defo3.merge(defo4).map(set_defo)

collection2 = fire2.merge(drought2).merge(stable2).merge(defo2).map(set_id).randomColumn(seed=3)

##### Classification and cross validation

In [ ]:
def classification(c, p):
    val0 = ee.Number(c)
    val1 = val0.add(0.2)
    train = collection2.filter(ee.Filter.Or(ee.Filter.lte('random', val0), ee.Filter.gt('random', val1)))
    vali = collection2.filter(ee.Filter.And(ee.Filter.gt('random', val0), ee.Filter.lte('random', val1)))
    sample_train = inputImg.sampleRegions(collection=train,
                                          properties=['change'],
                                          scale=30,
                                          geometries=True)

    sample_vali = inputImg.sampleRegions(collection=vali,
                                         properties=['change'],
                                         scale=30,
                                         geometries=True)

    classifier = ee.Classifier.smileRandomForest(tree)
    trainedRF = classifier.train(sample_train, 'change', classBands)
    trainCM = trainedRF.confusionMatrix()
    validationDS = sample_vali.classify(trainedRF)
    validationCM = validationDS.errorMatrix('change', 'classification')
    classified = inputImg.classify(trainedRF)
    return ee.List(p).add(ee.List([trainCM.accuracy(), trainCM.kappa(), trainCM.fscore(), trainCM.consumersAccuracy(), trainCM.producersAccuracy(),
                               validationCM.accuracy(), validationCM.kappa(), validationCM.fscore(), validationCM.consumersAccuracy(), validationCM.producersAccuracy(),
                               classified]))


iter2 = values.iterate(classification, ee.List([])).getInfo()

In [ ]:
kappa_validation = [n[6] for n in iter2]
accuracy_validation = [n[5] for n in iter2]
print(np.mean(kappa_validation), np.mean(accuracy_validation))

0.8343669743755436 0.8911333603359127


##### Visualisation

In [ ]:
classBands = inputImg.bandNames()
classifier = ee.Classifier.smileRandomForest(tree)
sample_train = inputImg.sampleRegions(collection=collection2,
                                      properties=['change'],
                                      scale=30,
                                      geometries=True)
trainedRF = classifier.train(sample_train, 'change', classBands)
classified_img = inputImg.classify(trainedRF)

In [44]:
classified_img2 = ee.Image(0).where(classified_img.gt(0), classified_img).multiply(mask)
mapid_img = classified_img2.focal_mode(2).updateMask(classified_img2.gt(0)).getMapId({'min': 0, 'max': 3, 'palette':['00FF00', 'cc0013', 'cdb33b', '00FF00']})
centroid = collection.geometry().centroid().coordinates().getInfo()[::-1]
map = folium.Map(location=centroid, zoom_start=6)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid_img['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='defo',
  ).add_to(map)

map.add_child(folium.LayerControl())
map